# Pip Wheels

In [1]:
'''
!pip install pytorch_lightning
!pip install torchmetrics
!pip install tokenizers
!pip install transformers
!pip install ray[tune]
'''

'\n!pip install pytorch_lightning\n!pip install torchmetrics\n!pip install tokenizers\n!pip install transformers\n!pip install ray[tune]\n'

# Imports

In [2]:
# General Libraries
import os
import re
import random
import numpy as np
import pandas as pd
import scipy as sp



# PyTorch Lightning
import pytorch_lightning as pl
from pytorch_lightning import seed_everything, Trainer, LightningModule
from torchmetrics import Accuracy
from torchmetrics.functional import f1_score, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar 
from pytorch_lightning.loggers import TensorBoardLogger



# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


# Ray[Tune]
import ray
from ray import air
from ray import tune
from ray.air import session
from ray.tune.integration.pytorch_lightning import TuneReportCallback

import torch
# HuggingFace Libraries
import tokenizers
import transformers 

%env TOKENIZERS_PARALLELISM=true

#out code 
from kfold_loop import KFoldLoop
from USPPM_model import USPPPM_model
from USPPM_dataset import set_tokenizer, set_max_len
from USPPM_kfold_datamodule import USPPPM_kf_datamodule

from datetime import datetime



env: TOKENIZERS_PARALLELISM=true


In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Configuration

## Configuration Class: notebook-specific settings

In [4]:
class CFG:
    # General
    seed = 42
    
    # Debug 
    debug = False
    debug_samples = 100

## Configuration Dictionary: trial-specific settings

In [5]:
# Defining a search space!
config_dict = {
    "target_size" : 1,
    "num_workers" : 16,
    
    # Training parameters
    "batch_size" : tune.grid_search([4,8,16,32,64,128]),
    "epochs" : 2,
    "n_fold" : tune.grid_search([8,16,32]),
    "warmup_steps" : 0,
    "min_lr" : 1e-6,
    "encoder_lr" : 2e-5,
    "decoder_lr" : 2e-5,
    "eps" : 1e-6,
    "betas" : (0.9, 0.999),
    "weight_decay" : 0.01,
    "fc_dropout" : 0.2,
    "seed" : 42,

    # Transformers
    # "model" : tune.choice(["microsoft/deberta-v3-large"]),
    #"model" : tune.choice(["distilbert-base-uncased"]),
    "model" : tune.grid_search(["AI-Growth-Lab/PatentSBERTa","distilbert-base-uncased","ahotrod/electra_large_discriminator_squad2_512",
                                "Yanhao/simcse-bert-for-patent","microsoft/deberta-v3-large","anferico/bert-for-patents"])
}

## Directories

In [6]:
INPUT_DIR = '../dataset/us-patent-phrase-to-phrase-matching/'
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Data Loading

In [7]:
cpc_texts = torch.load('cpc_texts.pth')
dataframe = pd.read_csv("dataframe.csv")
display(dataframe.head())

,Unnamed: 0,id,anchor,target,context,score,context_text,text,score_map
0,0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]abatement of pollution[SEP]HUMAN...,2
1,1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]act of abating[SEP]HUMAN NECESSI...,3
2,2,36d72442aefd8232,abatement,active catalyst,A47,0.25,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]active catalyst[SEP]HUMAN NECESS...,1
3,3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]eliminating process[SEP]HUMAN NE...,2
4,4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[SEP]forest region[SEP]HUMAN NECESSIT...,0


## Debug Slicing

In [8]:
if CFG.debug:
    dataframe = dataframe.iloc[:CFG.debug_samples,:]

## Train-Test Split

In [9]:
# train_df, test_df = train_test_split(dataframe, test_size = 0.1, random_state = CFG.seed, stratify = dataframe.score_map)
train_df, test_df = train_test_split(dataframe, test_size = 0.1, random_state = CFG.seed)
display(train_df.head())
display(test_df.head())

,Unnamed: 0,id,anchor,target,context,score,context_text,text,score_map
9900,9900,0dbb44b9a145edec,distributor pipe,pipe,B01,0.50,PERFORMING OPERATIONS; TRANSPORTING. PHYSICAL ...,distributor pipe[SEP]pipe[SEP]PERFORMING OPERA...,2
1303,1303,74afca34a5439c23,ammonia recovery,recovery of water,C01,0.25,HEMISTRY; METALLURGY. INORGANIC CHEMISTRY,ammonia recovery[SEP]recovery of water[SEP]HEM...,1
16591,16591,6371befc3ee1b0f2,inner closed,cylindrical inner member,E04,0.50,FIXED CONSTRUCTIONS. BUILDING,inner closed[SEP]cylindrical inner member[SEP]...,2
25822,25822,20489196c73bd86b,produce thin layers,produce layers,G01,0.50,PHYSICS. MEASURING; TESTING,produce thin layers[SEP]produce layers[SEP]PHY...,2
23640,23640,9af994b21c892022,parallel orientation,zero angle,G06,0.25,PHYSICS. COMPUTING; CALCULATING; COUNTING,parallel orientation[SEP]zero angle[SEP]PHYSIC...,1


,Unnamed: 0,id,anchor,target,context,score,context_text,text,score_map
33511,33511,ed1c4e525eb105fe,transmit alarm,display indicator,G08,0.00,PHYSICS. SIGNALLING,transmit alarm[SEP]display indicator[SEP]PHYSI...,0
18670,18670,5386316f318f5221,locking formation,retaining element,B60,0.25,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,locking formation[SEP]retaining element[SEP]PE...,1
18049,18049,1544ca6753fcbddd,lateral power,transducer,H01,0.25,ELECTRICITY. BASIC ELECTRIC ELEMENTS,lateral power[SEP]transducer[SEP]ELECTRICITY. ...,1
31660,31660,f9d8979b94cec923,spreader body,spreader,A01,0.75,HUMAN NECESSITIES. GRICULTURE; FORESTRY; ANIMA...,spreader body[SEP]spreader[SEP]HUMAN NECESSITI...,3
15573,15573,e151ca5ea5cc0f08,high gradient magnetic separators,magnetic filtration,B03,0.50,PERFORMING OPERATIONS; TRANSPORTING. SEPARATIO...,high gradient magnetic separators[SEP]magnetic...,2


# Training

## Callbacks

In [10]:
ray.init(num_gpus=4)

2022-11-14 15:38:56,087	INFO worker.py:1518 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.0.1


In [11]:


metrics = {"val_score": "val_score", "train_loss" : "train_loss", "val_loss" : "val_loss"}

In [12]:
def trainable(config_dict):  # Pass a "config" dictionary into your trainable.
    trial_id = ray.air.session.get_trial_id()
    logging_dir = f"USPPPM_{trial_id}"
    
    export_path = f'./ensemble_checkpoints/{trial_id}'
    
    for d in ["ensemble_checkpoints/",export_path,"lightning_logs",f"lightning_logs/{logging_dir}"]:
        try:
            os.mkdir(d)
        except FileExistsError:
            pass
    
    logger = TensorBoardLogger("lightning_logs", name=logging_dir)
    pl.seed_everything(CFG.seed)
    
    checkpoint_callback = ModelCheckpoint(
    dirpath=f"checkpoints/{trial_id}_checkpoints",
    filename="best_checkpoint",
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
    )

    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)
    
    steps_per_epoch = len(train_df) // config_dict['batch_size']
    config_dict['training_steps'] = steps_per_epoch * config_dict['epochs']
    
    set_tokenizer(config_dict, OUTPUT_DIR)
    set_max_len(config_dict, cpc_texts, dataframe)
    # train_dataset = USPPM_dataset(config_dict)
    datamodule = USPPPM_kf_datamodule(config_dict, dataframe)
    
    model = USPPPM_model(config_dict)
    
    callbacks = [TuneReportCallback(metrics, on="validation_end"), checkpoint_callback, early_stopping_callback, TQDMProgressBar(refresh_rate=2)]
    trainer = pl.Trainer(
            logger=logger,
            num_sanity_val_steps=0,
            check_val_every_n_epoch=1,
            callbacks=callbacks,
            max_epochs=config_dict['epochs'],
            #devices=[1],
            accelerator="gpu",
            )
    

    
    internal_fit_loop = trainer.fit_loop
    trainer.fit_loop = KFoldLoop(config_dict['n_fold'], config_dict, export_path=export_path)
    trainer.fit_loop.connect(internal_fit_loop)
    
    trainer.fit(model, datamodule)

In [13]:
resource_group = tune.PlacementGroupFactory([{"CPU": 1, "GPU": 1}])

tuner = tune.Tuner(tune.with_resources(trainable, 
                                       {"cpu":0.25,"gpu":1}),
                                       param_space = config_dict,
                                       tune_config = tune.TuneConfig(metric="val_score", mode="max",max_concurrent_trials=4),
                                       # tune_config = tune.TuneConfig(metric="val_score", mode="max"),
                                       run_config = air.RunConfig(name="tune_uspppm", verbose=2, progress_reporter=tune.JupyterNotebookReporter(overwrite=True))
                                       )


Trial name,status,loc,batch_size,model,n_fold
trainable_13334_00000,RUNNING,131.114.50.210:4032152,4,AI-Growth-Lab/P_5850,8
trainable_13334_00001,RUNNING,131.114.50.210:4032261,8,AI-Growth-Lab/P_5850,8
trainable_13334_00002,RUNNING,131.114.50.210:4032435,16,AI-Growth-Lab/P_5850,8
trainable_13334_00003,RUNNING,131.114.50.210:4033973,32,AI-Growth-Lab/P_5850,8


In [ ]:
results = tuner.fit()

best_result = results.get_best_result()  # Get best result object
print(best_result)

2022-11-14 15:38:59,024	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
(trainable pid=4032152) Global seed set to 42
100%|██████████| 36473/36473 [00:02<00:00, 13455.56it/s]
(trainable pid=4032261) Global seed set to 42
 78%|███████▊  | 28582/36473 [00:01<00:00, 15249.80it/s]
(trainable pid=4032152) GPU available: True (cuda), used: True
(trainable pid=4032152) TPU available: False, using: 0 TPU cores
(trainable pid=4032152) IPU available: False, using: 0 IPUs
(trainable pid=4032152) HPU available: False, using: 0 HPUs
(trainable pid=4032152) /storagenfs/m.petix/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:229: LightningDeprecationWarning: The `on_init_start` callback hook was deprecated in v1.6 and will be removed in v1.8.
(trainable pid

(trainable pid=4032152) STARTING FOLD 1
(trainable pid=4032152) TRAIN FOLD 1 28721
(trainable pid=4032152) VALID FOLD 1 4104
Epoch 0:   0%|          | 0/8207 [00:00<?, ?it/s] 


100%|██████████| 36473/36473 [00:02<00:00, 15199.12it/s]
(trainable pid=4032435) Global seed set to 42


Epoch 0:   0%|          | 4/8207 [00:01<1:02:24,  2.19it/s, loss=0.65, v_num=0, train_loss=0.713]


  4%|▍         | 1623/36473 [00:00<00:02, 16221.55it/s]


...

Epoch 0:   2%|▏         | 164/8207 [00:17<14:09,  9.47it/s, loss=0.593, v_num=0, train_loss=0.625]


(trainable pid=4033973) GPU available: True (cuda), used: True
(trainable pid=4033973) TPU available: False, using: 0 TPU cores
(trainable pid=4033973) IPU available: False, using: 0 IPUs
(trainable pid=4033973) HPU available: False, using: 0 HPUs
(trainable pid=4033973) /storagenfs/m.petix/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:229: LightningDeprecationWarning: The `on_init_start` callback hook was deprecated in v1.6 and will be removed in v1.8.
(trainable pid=4033973)   rank_zero_deprecation(
(trainable pid=4033973) /storagenfs/m.petix/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:233: LightningDeprecationWarning: The `on_init_end` callback hook was deprecated in v1.6 and will be removed in v1.8.
(trainable pid=4033973)   rank_zero_deprecation("The `on_init_end` callback hook was deprecated in v1.6 and will be removed in v1.8.")
(trainable pid=4033973) /storagenfs/m.petix/.local/lib/python3.8/s

Epoch 0:   2%|▏         | 190/8207 [00:19<13:59,  9.55it/s, loss=0.66, v_num=0, train_loss=0.626] 


(trainable pid=4033973) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [2]
(trainable pid=4033973) 
(trainable pid=4033973)   | Name       | Type              | Params
(trainable pid=4033973) -------------------------------------------------
(trainable pid=4033973) 0 | model      | MPNetModel        | 109 M 
(trainable pid=4033973) 1 | criterion  | BCEWithLogitsLoss | 0     
(trainable pid=4033973) 2 | fc_dropout | Dropout           | 0     
(trainable pid=4033973) 3 | fc         | Linear            | 769   
(trainable pid=4033973) 4 | attention  | Sequential        | 394 K 
(trainable pid=4033973) -------------------------------------------------
(trainable pid=4033973) 109 M     Trainable params
(trainable pid=4033973) 0         Non-trainable params
(trainable pid=4033973) 109 M     Total params
(trainable pid=4033973) 439.526   Total estimated model params size (MB)


Epoch 0:   2%|▏         | 192/8207 [00:20<13:58,  9.56it/s, loss=0.654, v_num=0, train_loss=0.500]
(trainable pid=4033973) STARTING FOLD 1
(trainable pid=4033973) TRAIN FOLD 1 28721
(trainable pid=4033973) VALID FOLD 1 4104
Epoch 0:  26%|██▌       | 2140/8207 [03:29<09:52, 10.23it/s, loss=0.565, v_num=0, train_loss=0.376]


In [ ]:
ray.shutdown()

In [ ]:
# Get a dataframe for the last reported results of all of the trials 
df = results.get_dataframe() 

In [ ]:
df.to_csv('grid_search_results.csv')